In [1]:
function getMotif(DNA, ind, k, t)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    return sample
end

getMotif (generic function with 1 method)

In [2]:
function Score(DNA, t, k, ind)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    score = 0
    for i=1:k
        count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j=1:t
            count[sample[j][i]] = count[sample[j][i]] + 1
        end
        max = 0
        for each in count
            if each[2]>max
                max = each[2]
            end
        end
        score = score+max
    end
    return score/k
end

Score (generic function with 1 method)

In [3]:
function decreaseKbest(NDSArchive)
    lastValue = length(NDSArchive[end-1]) 
    return lastValue
end

decreaseKbest (generic function with 1 method)

In [17]:
function crowdingDist(objs)
    crowdDist = zeros(Float64, size(objs)[2])
    for i in 1:size(objs)[1]
        v = sortperm(objs[i,:], rev=true)
        crowdDist[v[1]] = crowdDist[v[1]] + 2*objs[i,v[1]]
        crowdDist[v[end]] = crowdDist[v[end]] + 2*objs[i,v[end]]
        for j in 2:length(crowdDist)-1
            crowdDist[v[j]] = crowdDist[v[j]] + (objs[i,v[j-1]] + objs[i,v[j+1]])
        end
    end
    return crowdDist
end

crowdingDist (generic function with 1 method)

In [5]:
function nds(metric)
    ndsArchive = Array{Int64,1}[]
    dominated = zeros(Int64, length(metric))
    dominates = Array{Int64}[]
    tempnds = []
    for i in 1:length(metric)
        tempDom = []
        for j in 1:length(metric)
            if metric[i] > metric[j]
                push!(tempDom, j)
            elseif metric[i] < metric[j]
                dominated[i] = dominated[i] + 1
            end
        end
        push!(dominates, tempDom)
        if dominated[i] == 0
            push!(tempnds, i)
        end
    end
    push!(ndsArchive, tempnds)
    i = 1
    while length(ndsArchive[i]) != 0
        tempndsnext = []
        tempndscurr = ndsArchive[i]
        for j in 1:length(tempndscurr)
            p = tempndscurr[j]
            currSet = dominates[p]
            for k in 1:length(currSet)
                q = currSet[k]
                dominated[q] = dominated[q] - 1
                if dominated[q] == 0
                    push!(tempndsnext, q)
                end
            end
        end
        push!(ndsArchive, tempndsnext)
        i = i + 1
    end
    return ndsArchive
end

nds (generic function with 1 method)

In [6]:
function getNucProb(DNA)
    count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
    n = length(DNA[1])
    t = length(DNA)
    for i in 1:t
        for j in 1:n
            count[DNA[i][j]] = count[DNA[i][j]] + 1
        end
    end
    countN = []
    for each in count
        push!(countN, each[2])
    end
    countSum = sum(countN)
    prob = countN ./ countSum
    return prob
end

getNucProb (generic function with 1 method)

In [7]:
function complexity(DNA, t, k, ind, prob)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    den = 1
    consensusDict = zeros(Int64, 4, 1)
    for i=1:k
        count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j=1:t
            count[sample[j][i]] = count[sample[j][i]] + 1
        end
        countN = []
        for each in count
            push!(countN, each[2])
        end
        max = 0    
        consensus = i
        for i in 1:4
            if countN[i] > max
                max = countN[i]
                consensus = i
            end
        end
        consensusDict[consensus] = consensusDict[consensus] + 1
    end
    consensusDict = consensusDict .+ 1
    consensusDict = consensusDict ./ (k+4)
    complexityScore = 0
    for i in 1:4
        temp = log(2, consensusDict[i]/prob[i]) * consensusDict[i]
        complexityScore = complexityScore + temp
    end
    return complexityScore
end

complexity (generic function with 1 method)

In [8]:
function generateInitialPopulation(DNA,k, t, N)
    startIndices = Array{Int64}(undef,t,N)
    for i=1:t
        for j=1:N
            sIndex = rand(1:length(DNA[i])-k[j]-1)
            startIndices[i, j] = sIndex
        end
    end
    return startIndices
end

generateInitialPopulation (generic function with 1 method)

In [9]:
function genIndividual(t, ub)
    M = zeros(Int64, 1, t)
    s = rand(t)*(ub)
    for i in 1:t
        s[i] = s[i] + 1
        M[i] = trunc(Int64, s[i])
    end
    return M
end

genIndividual (generic function with 1 method)

In [10]:
function extract(archive)
    temp = []
    for each in archive
        append!(temp, each)
    end
    result = Array{Int64, 1}(undef, length(temp))
    for i in 1:length(temp)
        result[i] = temp[i]
    end
    return result
end

extract (generic function with 1 method)

In [24]:
using LinearAlgebra

function gsa(DNA, t, N, timeLimit =100)
    ndsArchive = Array{Int64,1}[]
    currentTime = 0
    kBest = N
    n = length(DNA[1])
    f = zeros(Float64, t, N)
    acc = Array{Float64}(undef, t, N)
    k = zeros(Int64, 1, N)
    m = zeros(Float64, 1, N)
    M = zeros(Float64, 1, N)
    fitness = zeros(Float64, 1, N)
    comp = zeros(Float64, 1, N)
    for i=1:N
        k[i] = rand(7:64)
    end
    #print(k)
    x = generateInitialPopulation(DNA,k, t, N)
    G = 100
    alpha = 20
    epsilon = 0.1
    
    while(currentTime < timeLimit)
        #print(currentTime)
        #println(x)
        for i=1:N
            fitness[i] = Score(DNA, t, k[i], x[1:t,i])
        end
        prob = getNucProb(DNA)
        for i=1:N
            comp[i] = complexity(DNA, t, k[i], x[1:t,i], prob)
        end
        
        objs = zeros(Float64, 3, N)
        #println(prob)
        #println(comp)
        #println(k ./ 64)
        objs[1,:] = k ./ 64
        objs[2,:] = fitness
        objs[3,:] = comp
        
        #println(objs[3,:])
        
        p = crowdingDist(objs)
        ndsArchive = nds(p)
        ndsArchiveExtract = extract(ndsArchive)
        #println(p)
        
        #Updating G, Best and Worst
        G = G * exp((-1 * alpha * currentTime)/timeLimit)
        best = maximum(fitness)
        worst = minimum(fitness)
        #println(m)
        #println(best, ' ', worst)
        #Calculate mass of each individual
        for i=1:N
            m[i] = (fitness[i])/(best)
        end
        #println(m)
        sumOfMass = sum(m)
        for i=1:N
            M[i] = m[i]/sumOfMass
        end
        #println(M)
        #Calculate force and acceleration on each individual 63
        for d=1:t
            fTemp = zeros(Float64,N,N)
            for i=1:N
                for j=1:kBest
                    r = norm(x[d,i]-x[d,j])
                    fTemp[i,j] = (G * ((M[i] * M[j])/(r + epsilon)) * (x[d,i]-x[d,j]))
                end
                #println(fTemp)
                for j in kBest
                    if j!=i
                       f[d,i] = f[d,i] + fTemp[i,j] 
                    end
                end
                acc[d,i] = f[d,i]/(M[i])
            end
        end
        #Calculate velocity and position of each individual
        #print(vel)
        #println(m)
        vel = zeros(Float64, t, N)
        for d=1:t
            for i=1:N
                #vel[d,i] = rand(0:1) * vel[d,i] + acc[d,i]
                if acc[d,i] > 0
                    vel[d,i] = acc[d,i]*(n - x[d,i])/sum(acc[d,:])
                else
                    vel[d,i] = acc[d,i]*(x[d,i]-1)/sum(acc[d,:])
                end
                tempXdi = convert(Int, floor(x[d,i] + vel[d,i]))
            end
        end
        k[:] = k[ndsArchiveExtract]
        fitness[:] = fitness[ndsArchiveExtract]
        comp[:] = comp[ndsArchiveExtract]
        for i in 1:size(x)[1]
            x[i,:] = x[i,:][ndsArchiveExtract]
        end
        kBest = trunc(Int64, N/2)
        #println(ndsArchive)
        #println(kBest)
        #println(currentTime)
        #println(ndsArchive)
        currentTime = currentTime + 1
    end
    #println(size(x))
    #Print Motifs
    println(transpose(x[:,ndsArchive[1]]))
    println(ndsArchive[1])
    println(k[ndsArchive[1]][1])
    println(Score(DNA, t, k[ndsArchive[1]][1], transpose(x[:,ndsArchive[1]])))
    return getMotif(DNA, transpose(x[:,ndsArchive[1]]), k[ndsArchive[1]][1], t)
end

gsa (generic function with 2 methods)

In [25]:
open("test1.txt") do file
    x = readline(file)
    line1 = split(x," ")
    t = parse(Int64, line1[1])
    DNA = Array{String,1}(undef,t)
    for i in 1:t
        DNA[i] = readline(file)
    end
    @time Motif1 = gsa(DNA, t, 200,250)
    for each in Motif1
        println(each)
    end
    println()
end

[63 110 92 100 26]
[1]
63
3.1904761904761907
  2.311979 seconds (19.18 M allocations: 3.029 GiB, 13.47% gc time)
CGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCGGGGCGGGACTTACCCCC
CGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCTCCCGCCCCCCACCCCTCG
CGGGCGCAGGGCCGGCGGGCCGCGCGCCCGTAAGGGGGGCCCCCCGGTGAGGAGGGCCCGTGC
CACCCCCAATCCCGCCCGGGGGGGGCCCTCAGTCCGAGCCCCCGACCCCCGCCGCTCCAGCCC
GGCCCTGGCGGGGGGGGGATGACTCTTCCACGGGTCGCCAGGGCGGCGAGGGCGCGTCGGGCC



In [316]:
?sign

search: sign signed Signed signbit significand Unsigned unsigned flipsign



```
sign(x)
```

Return zero if `x==0` and $x/|x|$ otherwise (i.e., ±1 for real `x`).


gsa (generic function with 2 methods)

In [21]:
#GCTACACAGTACATGGACATGCGGA